## Use the API to create some test data


In [ ]:
# Local Docker
auth_url = 'http://web:8000/api-token-auth/'
participant_url = 'http://web:8000/participant/'
submission_url = 'http://web:8000/submission/'

In [ ]:
# AWS (demo)
auth_url = 'https://researchnet.ictedge.org/api-token-auth/'
participant_url = 'https://researchnet.ictedge.org/participant/'
submission_url = 'https://researchnet.ictedge.org/submission/'

In [1]:
# Local
auth_url = 'http://127.0.0.1:8000/api-token-auth/'
participant_url = 'http://127.0.0.1:8000/participant/'
submission_url = 'http://127.0.0.1:8000/submission/'

In [2]:
import requests
payload = {'username': 'researcher', 'password':'1234thumbwar'}
r = requests.post(auth_url, json=payload)


In [3]:
r.json()['token']

'fc5c0358550770c9e9e5f7b9a865070104488336'

### Make up some stuff

In [4]:
import random

first_names = ['Pat', 'Jessie', 'Same', 'Sue', 'Chris', 'Piper']
last_names = ['Smith', 'Jones', 'Roberts', 'Walters', 'Thomas']
gender_options = ['Male', 'Female']

# lat/long
fake_places = [
     ("40.5895466", "-105.0751243"),
     ("41.5895466", "-104.0751243"),
     ("39.7640021","-105.1353041"),
     ("38.7640021","-104.1353041"),
     ("39.7724466","-105.9249464"),
     ("35.7724466","-103.9249464"),
     ("39.1490189","-107.1444289"),
     ("38.1490189","-106.1444289"),
     ("36.0017455","-79.0249976"),
     ("37.0017455","-80.0249976"),
     ("35.9661857","-79.1067084"),
     ("34.9661857","-78.1067084"),
     ("35.8436867","-78.7851426"),
     ("34.8436867","-75.7851426"),
     ("45.5423508","-122.7945071"),
     ("44.5423508","-121.7945071")
    ]


In [5]:
from random import randrange
from datetime import timedelta, datetime

def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [6]:
d1 = datetime.strptime('1/1/1967', '%m/%d/%Y')
d2 = datetime.strptime('1/1/1987', '%m/%d/%Y')

## Load Enrollments
The step step here is to generate some sample data.  Feel free to adjust this as needed.

In [7]:
for i in range(10):
    random_birthday = random_date(d1, d2)
    payload = {'username': 'testuser'+str(i), 
               'password':'testpassword', 
               'first_name': random.choice(first_names),
               'last_name': random.choice(last_names),
               'email': 'fakeemail@rti.org',
               'dob': str(random_birthday.strftime('%Y-%m-%d')),
               'gender': random.choice(gender_options),
              }
    
    r = requests.post(participant_url, json=payload)
 

## Load Submissions

In [8]:
# Randomize test data (its more interesting this way)
moods_category = ['good', 'bad', 'fine', 'unsure', 'no response', 'asleep']
mood_scale = ['1','2','3','4','5','6','7','8','9','0']
mood_image = ['1','2','3','4','5','6','7','8','9','0']

In [11]:
import random

# loop through the same participants loaded earlier
for i in range(10):
    
    # Get the token
    auth_payload = {'username': 'testuser'+str(i), 'password':'testpassword'}
    r_auth = requests.post(auth_url, json=auth_payload)
    auth_token = r_auth.json()['token']
    
    for y in range(random.randrange(1, 10, 2)):
        headers = {'Authorization': 'Token '+auth_token }
        submission_payload = {'device_id': 'TEST-49D5-4CAD-AE42-E5CE922A3346', 
           'lat' : random.choice(fake_places)[0],
           'long': random.choice(fake_places)[1],                
           'response': { 
                'mood cateogry': random.choice(moods_category),
                'mood scale': random.choice(mood_scale),
                'mood image': random.choice(mood_image)
            }
        }
     
        r_sub = requests.post(submission_url, json=submission_payload, headers=headers)

In [19]:
r_sub.text

'{"id":82,"user":"testuser9","time_start":"2016-03-30T14:54:18.925491","time_complete":"2016-03-30T14:54:18.925502","timestamp":"2016-03-30T20:54:18.926070Z","device_id":"TEST-49D5-4CAD-AE42-E5CE922A3346","response":{"mood scale":"4","mood image":"8","mood cateogry":"fine"},"lat":39.1490189,"long":-103.9249464}'

In [14]:

import geocoder
g = geocoder.google([40.5895466, -105.0751243], method='reverse')
g.city

'Fort Collins'